# Introduction to the `complexity-science` package
## and some HPC tips

## Basic Usage

In [ ]:
import complexity_science.ca as ca

In [ ]:
import complexity_science as cx

# Available Models

## 1D Models
- ca.wolfram

## 2D Models
- ca.game, ca.brians_brain, ca.mpa, ca.applause, ca.forest_fire

## CA cheatsheet
- demo help
- demo autofill
- Classes vs objects/models/methods

In [ ]:
model = ca.wolfram(200, 90)

In [ ]:
help(ca.wolfram)

In [ ]:
ca.wolfram?

In [ ]:
model.

## 1D CA
##### Wolfram

In [ ]:
ca90 = ca.wolfram(200, 90) #RULE 90

In [ ]:
ca90.initialize_random()
# ca90.cells

In [ ]:
ca90.initialize_index([50,150])
results = ca90.run(100)

# 2D CA

In [ ]:
model = ca.game([128, 128], toroidal=False)

In [ ]:
help(model.initialize_random)

In [ ]:
model.initialize_random_bin(0.8)

In [ ]:
help(model.evolve)

In [ ]:
t1 = model.evolve()
t1

## Animation

In [ ]:
model = ca.game([128, 128], toroidal=False)
model.initialize_random_bin(0.5)

In [ ]:
# Use outside jupyter notebook
model.animate()

## Animation on jupyter notebook

In [ ]:
from IPython.display import HTML

In [ ]:
anim = model.jyp_anim(cmap='plasma')

In [ ]:
HTML(anim.to_html5_video())

## Other examples

In [ ]:
help(ca.mpa)

In [ ]:
mpa_model = ca.mpa([100,100], percent_mpa=0.4)

In [ ]:
mpa_model.initialize_random()
mpa_anim = mpa_model.jyp_anim(cmap='viridis') #Minor issue: Shows final plot of animation

In [ ]:
HTML(mpa_anim.to_html5_video())

# BREAK

https://www.youtube.com/watch?v=xP5-iIeKXE8

Any questions?

UP NEXT: Creating your own model

# Creating new models
#### Base models
##### 1D (CA1D)
- CA, CA_nt, SimpleCA

#### 2D (CA2D)
- MooreCA_t, MooreCA_nt, VonCA_t, VonCA_nt

#### 3D
- CA3D

In [ ]:
new_model = ca.CA1D

In [ ]:
new_model = ca.VonCA([100,100])
new_model.initialize_random()

In [ ]:
new_model.cells

# new_model.cells = np.random.randint(0,5, [100,100])

In [ ]:
new_model.neighbors['left']

# Game of Life sample-code
![](cs_gol.png)

# MHPC basics

In [ ]:
import numpy as np
import timeit

In [ ]:
N = 1000
random1 = np.random.random([N,N])
random2 = np.random.random([N,N])

In [ ]:
def for_loop(xdim=N, ydim=N):
    total= np.zeros([N,N])
    for i in range(xdim):
        for j in range(ydim):
            total[i,j] = random1[i,j]+random2[i,j]
def vector_add():
    total = random1+random2

def numpy_sum():
    total = np.add(random1,random2)

In [ ]:
%timeit for_loop()

In [ ]:
%timeit vector_add()

In [ ]:
%timeit numpy_sum()

# HPC tips

1. Python for loops are slow
2. Branches slow down codes (if-else)
3. Vectorize if possible
4. Utilize compiled packages

# Game of life (vectorized)

In [ ]:
class GameOfLife:
    def apply(self, current, neighbors): #Parameters are important
        total_neighbors = np.zeros_like(current)

        for key in neighbors:
            total_neighbors += neighbors[key]

        result = np.zeros_like(current)
        state = current.copy()

        live = (current==1)
        dead = (current==0)
        two = (total_neighbors == 2)
        three = (total_neighbors ==3)

        less_two = (total_neighbors < 2)
        more_three = (total_neighbors >3)
        two_or_three = np.logical_or(two,three)

        result += np.logical_and(live, two_or_three).astype(int)
        result += np.logical_and(dead, three).astype(int)

        state -= np.logical_and(live, less_two).astype(int)
        state -= np.logical_and(live, more_three).astype(int)

        result = np.logical_or(result, state).astype(int)

        return result

# Customized Rules (Ask for suggestion)
mpa_anim = mpa_model.jyp_anim(cmap='viridis')

HTML(mpa_anim.to_html5_video())

# Customized Rules (Part2)

In [ ]:
class Reinier:
    def apply(self, current, neighbors): #Parameters are important
        a0 = 0
        a1 = 0.2
        a2 = 0.7
        
        average = np.zeros_like(current)
        
        for key in neighbors:
            average += neighbors[key]
            
        average = average/8
        
        
        rule1 = (average > a0).astype(int)
        rule2 = (average < a1).astype(int)
        
        within_range = (np.logical_and(rule1,rule2))
        
        a_out = a2*(average-a0)/(a1-a0)
        
        result = a_out*within_range
        return result

In [ ]:
new_model = ca.MooreCA_t([1000,1000])
new_model.add_rule(Reinier()) #or set_rule()
new_model.initialize_random()

In [ ]:
anim = new_model.jyp_anim(cmap='plasma')

In [ ]:
HTML(anim.to_html5_video())

# Data collection

In [ ]:
help(new_model.run_collect)

In [ ]:
data = new_model.run_collect(100)#, collector={'average':np.mean, 'max': np.max})

In [ ]:
data = new_model.run_collect(100, collector={'average':np.mean, 'max': np.max})

In [ ]:
data

# Contributing

1. Go to https://github.com/KristerJazz/complexity-science

## Parametrization

In [ ]:
mpa_model = ca.mpa([100,100], percent_mpa=0.4)
mpa_model.initialize_random()

In [ ]:
mpa_model.modify_rule(gamma=0.2)#, D=100, dt=0.001)

In [ ]:
mpa_anim = mpa_model.jyp_anim(cmap='viridis')
HTML(mpa_anim.to_html5_video())

In [ ]:
HTML(mpa_anim.to_html5_video())

# Parametrization

In [ ]:
class Rule:
    def apply(self, current, neighbor):
        pass
    
    def modify_rule(**params):
        pass

In [ ]:
import numpy as np
from .ca.ca_network import *
from .rules_network.migration import *
from .rules_network.gillespie import *

def gillespie(adj, heterogeneity=4, rule='default'): #SIRC
    model = CA_Network(adj, heterogeneity)
    if rule=='default':
        model.set_rule(SIRC_Gillespie()) #rule1
        model.add_rule(Migration()) #rule2
    return model

# def migration(adj):
#     model = CA_Network(adj,3)
#     model.set_rule(Migration())
#     return model